In [533]:
#Importing the required modules
import tensorflow as tf
import numpy as np

In [534]:
#Getting the Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot = True)

"""Now what is that one_hot = True?
 One-hot-encoding uses a vector of binary values to represent numeric or categorical values. 
 As our labels are for the digits 0-9, the vector contains ten values, one for each possible digit. 
 One of these values is set to 1, to represent the digit at that index of the vector, and the rest are set to 0. 
 For example, the digit 3 is represented using the vector [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]. As the value at index 3 is stored as 1, the vector therefore represents the digit 3"""

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


'Now what is that one_hot = True?\n One-hot-encoding uses a vector of binary values to represent numeric or categorical values. \n As our labels are for the digits 0-9, the vector contains ten values, one for each possible digit. \n One of these values is set to 1, to represent the digit at that index of the vector, and the rest are set to 0. \n For example, the digit 3 is represented using the vector [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]. As the value at index 3 is stored as 1, the vector therefore represents the digit 3'

In [535]:
print (mnist.train.num_examples)
print (mnist.train.images.shape)

55000
(55000, 784)


In [557]:
# Setting up hyper parameters (learning rate, training epochs, batch size, display step)

"""Let me first understand what each of the parameters mean

Learning Rate:
~~~~~~~~~~~~~~


Step size = Slope X Learning Rate
Practically, Gradient Descent stops when Slope is very close to 0 i.e 0.0001 so if slope is 0.009 X 0.001 = 0.00009 Which is smaller than our learning rate, so we stop

Way the learning rate changes from relatively large to relatively small is called the SCHEDULE
Basically Sum of squared residuals mei kitne se jump maarega
Deep learning neural networks are trained using the stochastic gradient descent algorithm.
(Stochastic GD randomly picks up one sample for each step and just use that one sample to calculate the derivates, Thus it reduces the calculations by a factor of n where n is the number of samples)
Stochastic gradient descent is an optimization algorithm that estimates the error gradient for the current state of the model using examples from the training dataset, then updates the weights of the model using the back-propagation of errors algorithm, referred to as simply backpropagation.
The amount that the weights are updated during training is referred to as the step size or the “learning rate.
A neural network learns or approximates a function to best map inputs to outputs from examples in the training dataset.
The learning rate hyperparameter controls the rate or speed at which the model learns. Specifically, it controls the amount of apportioned error that the weights of the model are updated with each time they are updated, such as at the end of each batch of training examples.
Given a perfectly configured learning rate, the model will learn to best approximate the function given available resources (the number of layers and the number of nodes per layer) in a given number of training epochs (passes through the training data).
Generally, a large learning rate allows the model to learn faster, at the cost of arriving on a sub-optimal final set of weights. A smaller learning rate may allow the model to learn a more optimal or even globally optimal set of weights but may take significantly longer to train.
At extremes, a learning rate that is too large will result in weight updates that will be too large and the performance of the model (such as its loss on the training dataset) will oscillate over training epochs. Oscillating performance is said to be caused by weights that diverge (are divergent). A learning rate that is too small may never converge or may get stuck on a suboptimal solution.

For Big Data, Gradient Descent is slow, this is where stochastic Gradient descent comes in handy

Training Epochs:
~~~~~~~~~~~~~~~~
The number of iterations to train the Stochastic Gradient Descent

Batch Size
~~~~~~~~~~~
The strict definition of Stochastic Gradient Descent is to take one sample each step.
However, it is more common to select mini batch for each step. Thus this parameter becomes the batch size.
”"""
learning_rate = 0.01
training_epochs = 100   #Good practice to set higher
batch_size = 256
display_step = 10

In [558]:
# setup network parameters (size of different hidden layers, size of input features, size of output classes)
n_hidden_1 = 256
n_hidden_2 = 256
n_hidden_3 = 256

"""Since input is of 28x28 images, we have total 784 pixel values"""
n_input = 784 

"""Each of the image can only belong to a class from 0 to 9"""
n_classes = 10

In [559]:
# setup input placeholders for X with shape of (None initial, n_input ) and Y with shape (None initial , n_classes )

"""To build our network, we will set up the network as a computational graph for TensorFlow to execute. 
The core concept of TensorFlow is the tensor, a data structure similar to an array or list. 
initialized, manipulated as they are passed through the graph, and updated through the learning process.

We’ll start by defining tensors as placeholders, which are tensors that we'll feed values into later. Add the following to your file:
The only parameter that needs to be specified at its declaration is the size of the data we will be feeding in. 
For X we use a shape of [None, 784], where None represents any amount, as we will be feeding in an undefined number of 784-pixel images. 
The shape of Y is [None, 10] as we will be using it for an undefined number of label outputs, with 10 possible classes. 
"""

X = tf.placeholder(tf.float32, shape=[None, n_input])
Y = tf.placeholder(tf.float32, shape=[None, n_classes])

In [560]:
# setup weight and bias variables
"""
The parameters that the network will update in the training process are the weight and bias values, so for these we need to set an initial value rather than an empty placeholder. 
These values are essentially where the network does its learning, as they are used in the activation functions of the neurons, representing the strength of the connections between units.

Since the values are optimized during training, we could set them to zero for now. But the initial value actually has a significant impact on the final accuracy of the model. 
We'll use random values from a truncated normal distribution for the weights. 
We want them to be close to zero, so they can adjust in either a positive or negative direction, and slightly different, so they generate different errors. 
This will ensure that the model learns something useful

"""
W = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='h1'),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='h2'),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3]), name='h3'),
    'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes]), name='w_out')
}

b = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='b1'),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='b2'),
    'b3': tf.Variable(tf.random_normal([n_hidden_3]), name='b3'),
    'out': tf.Variable(tf.random_normal([n_classes]), name='b_out')
}

In [561]:
# create a function name multilayer_perceptron(x,weights, biases) and return out_layer

def multilayer_perceptron(x, weights, bias):
    # 1 hidden layer with Linear and then RELU Activation
    layer1 = tf.add(tf.matmul(x, weights['h1']), bias['b1'])
    layer1 = tf.nn.relu(layer1)
    
    # 2 hidden layer with Linear and then RELU Activation
    layer2 = tf.add(tf.matmul(layer1, weights['h2']), bias['b2'])
    layer2 = tf.nn.relu(layer2)
    
    layer3 = tf.add(tf.matmul(layer1, weights['h3']), bias['b3'])
    layer3 = tf.nn.relu(layer3)


    
    # Outut layer with Linear Activation
    out_layer = tf.add(tf.matmul(layer3, weights['out']), bias['out'])
    
    return out_layer

In [562]:
# setup pred model
pred = multilayer_perceptron(X, W, b)

In [566]:
# setup cost which is average of cross entropy function aka -ylog(pred)
"""The final step in building the graph is to define the loss function that we want to optimize. 
A popular choice of loss function in TensorFlow programs is cross-entropy, also known as log-loss, which quantifies the difference between two probability distributions (the predictions and the labels). 
A perfect classification would result in a cross-entropy of 0, with the loss completely minimized.

We also need to choose the optimization algorithm which will be used to minimize the loss function. 
A process named gradient descent optimization is a common method for finding the (local) minimum of a function by taking iterative steps along the gradient in a negative (descending) direction. 
There are several choices of gradient descent optimization algorithms already implemented in TensorFlow, and in this tutorial we will be using the Adam optimizer. 
This extends upon gradient descent optimization by using momentum to speed up the process through computing an exponentially weighted average of the gradients and using that in the adjustments."""

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y))

# setup optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [567]:
# initialzie all variables
init = tf.global_variables_initializer()

In [568]:
# run the graph within session

with tf.Session() as sess:
    sess.run(init)
    
    # loop through all the epochs
    for epoch in range(training_epochs):
        avg_cost = 0
        
        #get total batches and loop through them
        total_batches = int(mnist.train.num_examples / batch_size)
        for batch in range(total_batches):
            
            # get trainX and trainY for each batch from training data
            trainX, trainY = mnist.train.next_batch(batch)
            
            # Train & run optimzer, cost in same session and feed values to Variables
            o, c = sess.run([optimizer, cost], feed_dict={X: trainX, Y:trainY})
            
            #compute average cost
            avg_cost += c / total_batches
            
            if (epoch % display_step) == -1:
                print ('batch=', batch+1, 'Cost=', c, 'Average Cost=', avg_cost, ' for Total batches', total_batches)
        
        # Test Model for every epoch
        correct_predection = tf.equal(tf.argmax(pred, axis=1), tf.argmax(Y, axis=1))
        
        # Calculate Accuracy aftering training
        accuracy_train = tf.reduce_mean(tf.cast(correct_predection, tf.float32))
        accuracy_eval_train = accuracy_train.eval({X: mnist.train.images, Y: mnist.train.labels})
        
        # Calculate Accuracy aftering testing
        accuracy_test = tf.reduce_mean(tf.cast(correct_predection, tf.float32))
        accuracy_eval_test = accuracy_test.eval({X: mnist.test.images, Y: mnist.test.labels})
        
        # print display step with epoch, and cost
        if (epoch % display_step) == 0 :
            print ('Epoch=', epoch+1, '  Train Accuracy=', accuracy_eval_train,'  Test Accuracy=', accuracy_eval_test)
        
    print("Optimizaton Finished for all epochs!")
    print ('Epoch=', epoch+1, 'Train Accuracy=', accuracy_eval_train, 'Test Accuracy=', accuracy_eval_test)
    # print optimization finished for all epochs

Epoch= 1   Train Accuracy= 0.8868545   Test Accuracy= 0.8828
Epoch= 11   Train Accuracy= 0.97294545   Test Accuracy= 0.9509
Epoch= 21   Train Accuracy= 0.98065454   Test Accuracy= 0.9562
Epoch= 31   Train Accuracy= 0.983   Test Accuracy= 0.9552
Epoch= 41   Train Accuracy= 0.96534544   Test Accuracy= 0.9469
Epoch= 51   Train Accuracy= 0.97005457   Test Accuracy= 0.9506
Epoch= 61   Train Accuracy= 0.9517091   Test Accuracy= 0.9396
Epoch= 71   Train Accuracy= 0.9556   Test Accuracy= 0.9462
Epoch= 81   Train Accuracy= 0.9762727   Test Accuracy= 0.9589
Epoch= 91   Train Accuracy= 0.9679818   Test Accuracy= 0.9525
Optimizaton Finished for all epochs!
Epoch= 100 Train Accuracy= 0.9774 Test Accuracy= 0.9615
